In [2]:
import pandas as pd
import os
import threading
from sklearn.ensemble import GradientBoostingRegressor
import altair as alt
import numpy as np
import yfinance as yf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

In [3]:
# https://www.kaggle.com/datasets

a_i_directory_path = r"C:\Users\Blake Dennett\Downloads\Spring2023\appliedProgramming\Data\stock_market_data\sp500\csv"
with_slash = r"C:\Users\Blake Dennett\Downloads\Spring2023\appliedProgramming\Data\stock_market_data\sp500\csv\'"

d_i_directory_path = r"C:\Users\Blake Dennett\Downloads\Spring2023\appliedProgramming\Data\stock_market_data\sp500\csv\d-i"
with_slash4 = r"C:\Users\Blake Dennett\Downloads\Spring2023\appliedProgramming\Data\stock_market_data\sp500\csv\d-i\'"

j_p_directory_path = r"C:\Users\Blake Dennett\Downloads\Spring2023\appliedProgramming\Data\stock_market_data\sp500\csv\j-p"
with_slash2 = r"C:\Users\Blake Dennett\Downloads\Spring2023\appliedProgramming\Data\stock_market_data\sp500\csv\j-p\'"

q_z_directory_path = r"C:\Users\Blake Dennett\Downloads\Spring2023\appliedProgramming\Data\stock_market_data\sp500\csv\j-p\q-z"
with_slash3 = r"C:\Users\Blake Dennett\Downloads\Spring2023\appliedProgramming\Data\stock_market_data\sp500\csv\j-p\q-z\'"

df = pd.DataFrame()
df2 = pd.DataFrame()
df3 = pd.DataFrame()
df4 = pd.DataFrame()
dataframes = []

In [4]:
# function to combine two datasets
def combine(dat1, dat2):
    to_combine = [dat1, dat2]
    return pd.concat(to_combine)


def get_company_name(id):
    msft = yf.Ticker(id)

    company_name = msft.info['longName']
    return company_name

In [5]:
# function to loop through and open files
def files_to_dataframe(directory_path, with_slash, df):
    for filename in os.listdir(directory_path):
        if filename == 'j-p'or filename == 'q-z' or filename == 'd-i':
            continue
        directory = with_slash[:-1]
        path = directory + filename
        with open(path, 'r') as file:
            dat = pd.read_csv(file)
        dat['company_id'] = filename[:len(filename)-4]
        df = combine(df, dat)
    dataframes.append(df)

# loops through the folder broken up into four threads that run concurently 
thread1 = threading.Thread(target=files_to_dataframe, args=(a_i_directory_path, with_slash, df))
thread2 = threading.Thread(target=files_to_dataframe, args=(j_p_directory_path, with_slash2, df2))
thread3 = threading.Thread(target=files_to_dataframe, args=(q_z_directory_path, with_slash3, df3))
thread4 = threading.Thread(target=files_to_dataframe, args=(d_i_directory_path, with_slash4, df4))

thread1.start()
thread2.start()
thread3.start()
thread4.start()
thread1.join()
thread2.join()
thread3.join()
thread4.join()

# combining the dataframes from each of the four threads
df = combine(dataframes[0], dataframes[1])
df2 = combine(dataframes[2], dataframes[3])
df = combine(df, df2)
df.dropna(inplace=True)
print(len(df))

# df['company'] = df.apply(get_company_name, axis=1)
# df.drop(columns=['company_id'])

3258423


## What is the biggest difference in the low and high?

In [6]:
# ==================================================== DATA ANALYSIS =========================================

df['difference'] = df.apply(lambda x: x.High - x.Low, axis=1)

max = df["difference"].max()
df.set_index("difference", inplace=True)

high = df.loc[max,'High']
low = df.loc[max,'Low']
company = get_company_name(df.loc[max,'company_id'])
date = df.loc[max, 'Date']

print(f"The biggest difference in high to low is {max} from {company}")
print(f'The high was {high:.2f} and the low was {low:.2f}')
print(f'The date was {date}')

df.reset_index(inplace=True)

The biggest difference in high to low is 53156.0 from Berkshire Hathaway Inc.
The high was 468795.00 and the low was 415639.00
The date was 27-10-2022


## What is the largest difference by percentage?

In [7]:
df['percent_difference'] = df.apply(lambda x: (x.difference / x.High) * 100, axis=1)
max = df['percent_difference'].max()

df.set_index("percent_difference", inplace=True)
company = df.loc[max, 'company_id']
high = df.loc[max, 'High']
low = df.loc[max, 'Low']
date = df.loc[max, 'Date']

print(f"The biggest difference by percent from high to low is {max} from {get_company_name(company)}")
print(f'The high was {high:.2f} and the low was {low:.4f}')
print(f'The date was {date}')

df.reset_index(inplace=True)
# print(df.head())

The biggest difference by percent from high to low is 99.98492462247178 from Formcap Corp.
The high was 1.99 and the low was 0.0003
The date was 23-11-2020


## What is the oldest and newest date?

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df['yr'] = pd.DatetimeIndex(df['Date']).year
oldest = df['Date'].min()
newest = df['Date'].max()
oldest = pd.to_datetime(oldest)
newest = pd.to_datetime(newest)
newest_company = df.loc[newest, 'company_id']
print(f'The oldest date is {oldest}')
print(f'The newest date is {newest}')

## Is the High and Low Based on the Year or Day?

In [ ]:
dfU = df.query('company_id == "UAL"')
dfU['month'] = pd.DatetimeIndex(dfU['Date']).month
dfU['day'] = pd.DatetimeIndex(dfU['Date']).day

dfU = dfU.query('(yr == 2006 & month == 7) & (day == 2 | day == 18)')

dfU.head()

The High and Low differ here by day, therefore, it seems to be the high and low by the day

# (get the year high and low), then get biggest difference in them

# Machine Learning to Guess the Close of Netflix

## Vanilla Tree

In [17]:
fidelity_df = df.query('company_id == "NFLX"')
fidelity_df.rename(columns={'difference':' difference', 'percent_difference':' percent_difference'}, inplace=True)
X = fidelity_df[[' difference', ' percent_difference', 'High', 'Low', 'Open', 'Volume']]
y = fidelity_df['Close']
random_states = [25, 42, 215, 190]
predictions = {}
for state in random_states:
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=state)
    reg = GradientBoostingRegressor(random_state=state)
    reg.fit(x_train, y_train)
    y_predictions = reg.predict(x_test)
    r2 = r2_score(y_test, y_predictions)
    import math
    rmse = math.sqrt(mean_squared_error(y_test, y_predictions))
    mae = mean_absolute_error(y_test, y_predictions)
    print(f'The r^2 is: {r2} and the RMSE is: {rmse}, and the MAE is {mae}')

C:\Users\Blake Dennett\AppData\Local\Temp\ipykernel_7552\1437283175.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fidelity_df.rename(columns={'difference':' difference', 'percent_difference':' percent_difference'}, inplace=True)


The r^2 is: 0.9997046006638671 and the RMSE is: 2.9158741404550104, and the MAE is 1.4077696954555095
The r^2 is: 0.9996684289389085 and the RMSE is: 3.041153588059448, and the MAE is 1.4191463768471408
The r^2 is: 0.9997338224399366 and the RMSE is: 2.7692515624697016, and the MAE is 1.3630224427109454
The r^2 is: 0.9997234994043414 and the RMSE is: 2.774196479926838, and the MAE is 1.3279887394202712


## Visualizations

In [ ]:

alt.data_transformers.disable_max_rows()
print(df.head())
id_df = df.query('Close < 50')
id_df.head()
format(len(id_df), "_d")

# Company Size Top Ten

In [ ]:
top_ten_df = df.query('Open > 100')
top_ten_df.drop(columns=['Date', 'Adjusted Close', 'Low', 'High', 'Close'], inplace=True)
top_ten_df['size'] = top_ten_df.apply(lambda x: round((x.Volume * x.Open) / 1000, 2), axis=1)
top_ten_df.head()
top_ten_df['company_id'].unique()

In [80]:
ids = {'QRVO':0, 'RCL':0, 'RE':0, 'REGN':0, 'RHI':0, 'RIBT':0, 'RJF':0, 'RL':0, 'RLI':0, 'RMD':0, 'ROK':0, 'ROP':0, 'ROST':0, 'RSG':0, 'SBUX':0, 'SHW':0, 'SIVB':0, 'SLB':0,
'SLG':0, 'SNPS':0, 'SPG':0, 'SRE':0, 'STT':0, 'STX':0, 'STZ-B':0, 'SWK':0, 'SWKS':0, 'SYK':0, 'TAP':0, 'TEL':0, 'TMO':0, 'TMUS':0, 'TROW':0, 'TRV':0, 'TSCO':0, 'TTWO':0,'TW':0, 'TXN':0, 'TYL':0, 'UHS':0, 'ULTA':0, 'UNP':0, 'UPS':0, 'URI':0, 'V':0, 'VMC':0,
'VRSK':0, 'VRSN':0, 'WAT':0, 'WDC':0, 'WEC':0, 'WHR':0, 'WM':0, 'WSPOF':0, 'WST':0, 'WYNN':0, 'XOM':0, 'XYL':0, 'YUM':0, 'ZBH':0, 'ZTS':0, 'A':0, 'AAP':0, 'AAPL':0, 'ABBV':0, 'ABC':0, 'ABMD':0, 'ABT':0, 'ACN':0, 'ADI':0, 'ADP':0, 'ADSK':0, 'AEP':0, 'AIZ':0, 'AJG':0, 'AKAM':0, 'ALB':0, 'ALGN':0, 'ALLE':0, 'AMAT':0, 'AMD':0, 'AME':0,
'AMGN':0, 'AMP':0, 'AMT':0, 'AMZN':0, 'ANET':0, 'AON':0, 'APA':0, 'APD':0, 'ARE':0, 'ATVI':0, 'AVB':0, 'AVY':0, 'AWK':0, 'AXP':0, 'AZO':0, 'BA':0, 'BBY':0, 'BDX':0,
'BIIB':0, 'BIO':0, 'BLK':0, 'BR':0, 'BRK-A':0, 'BXP':0, 'C':0, 'CAT':0, 'CB':0, 'CCI':0, 'CDE':0, 'CDNS':0, 'CF':0, 'CHD':0, 'CHRW':0, 'CHTR':0, 'CINF':0, 'CLX':0, 'CME':0, 'CMG':0, 'CMI':0, 'COO':0, 'COP':0, 'COST':0, 'CRM':0, 'CTAS':0, 'DE':0,
'DFS':0, 'DG':0, 'DGX':0, 'DHI':0, 'DIS':0, 'DLTR':0, 'DOV':0, 'DPZ':0, 'DRI':0, 'DTE':0, 'DVA':0, 'DXCM':0, 'EA':0, 'ECL':0, 'ED':0, 'EFX':0, 'EL':0, 'EMN':0, 'EMR':0, 'ENS':0, 'EOG':0, 'EQIX':0, 'ESS':0, 'EW':0, 'EXR':0, 'FANG':0, 'FBHS':0, 'FDX':0, 'FFIV':0, 'FIS':0, 'FISV':0, 'FLT':0, 'FMC':0, 'FN':0, 'FRC':0, 'FRT':0, 'GD':0,
'GE':0, 'GILD':0, 'GOOG':0, 'GPC':0, 'GPN':0, 'GRMN':0, 'GWW':0, 'HAS':0, 'HCA':0, 'HD':0, 'HES':0, 'HII':0, 'HLT':0, 'HON':0, 'HSY':0, 'HUM':0, 'IBM':0, 'ICE':0, 'IDXX':0, 'IEX':0, 'IFF':0, 'ILMN':0, 'INTH':0, 'INTU':0, 'IPGP':0, 'ISRG':0, 'IT':0,
'ITW':0, 'JBHT':0, 'JCI':0, 'JKHY':0, 'JNJ':0, 'JNPR':0, 'JPM':0, 'KEYS':0, 'KMB':0, 'KMX':0, 'LDOS':0, 'LH':0, 'LMT':0, 'LOW':0, 'LRCX':0, 'LVS':0, 'LYB':0, 'LYV':0, 'MAA':0, 'MAR':0, 'MCD':0, 'MCK':0, 'MCO':0, 'MDT':0, 'MHK':0, 'MKTX':0, 'MLM':0, 'MMC':0, 'MMM':0, 'MNST':0, 'MOS':0, 'MPC':0, 'MRK':0, 'MSCI':0, 'MSFT':0, 'MSI':0,
'NFLX':0, 'NOC':0, 'NOW':0, 'NOXL':0, 'NSC':0, 'NTAP':0, 'NTRA':0, 'NTRR':0, 'NTRS':0, 'NVR':0, 'NVRO':0, 'ODFL':0, 'ORLY':0, 'OXY':0, 'PAYX':0, 'PCAR':0, 'PEP':0, 'PG':0, 'PH':0, 'PKG':0, 'PKI':0, 'PLD':0, 'PM':0, 'PNW':0, 'PPG':0, 'PRU':0,'PSX':0, 'PVH':0, 'PWR':0, 'PXD':0}


for id, num in ids.items():
    new_df = top_ten_df.loc[id, 'size']
    new_df = pd.DataFrame(new_df)
    avg = new_df['size'].mean()
    ids[id] = avg

new_dict = {'id':[], 'size':[]}

for id, num in ids.items():

    if num > 1_470_000:
        new_dict['id'].append(id)
        new_dict['size'].append(num)

size_df = pd.DataFrame.from_dict(new_dict)

size_df['company'] = size_df['id'].apply(get_company_name)


size_df.drop(columns=['id'])

# REMOVE EVERYTHING UNDER 1 MILLION 5 HUNDRED THOUSAND, "SIVB" DOESNT WORK BUT I THINK ITS SMALL


,size,company
0,1.480765e+06,Visa Inc.
1,1.392002e+07,Apple Inc.
2,9.231353e+06,"Advanced Micro Devices, Inc."
3,1.151198e+07,"Amazon.com, Inc."
4,1.598853e+06,The Boeing Company
5,2.142385e+06,Berkshire Hathaway Inc.
6,3.292581e+06,Alphabet Inc.
7,1.626703e+06,"Juniper Networks, Inc."
8,1.704323e+06,JPMorgan Chase & Co.
9,6.072710e+06,Microsoft Corporation


In [92]:
size_chart = alt.Chart(size_df).mark_bar(color="orange").encode(
    x = alt.X('company', sort='-y', axis=alt.Axis(labelAngle=-45), title="Company"),
    y = alt.Y('size', title="Size (Avg Volume Times Open)"),
    text = 'size'
).properties(width=800, title="Top Companies")
size_chart

alt.Chart(...)

# Netflix Share Price Over Time

In [74]:
alt.data_transformers.disable_max_rows()
n_df = df.query('company_id == "NFLX"')
n_df['month'] = pd.DatetimeIndex(n_df['Date']).month


# get avg close by month, per year
frst_yr = n_df['yr'].min()
lst_yr = n_df['yr'].max()
yrs = [*range(frst_yr, lst_yr + 1)]

months = [*range(1, 12 + 1)]
n_df['avg_close_by_month'] = 0

for yr in yrs:
    for month in months:
        avg_df = n_df.query(f"(yr == {yr}) & (month == {month})")
        avg = avg_df['Close'].mean()
        n_df['Close'] = np.where(n_df['Close'] > 200, 1, 0) 


nflx_chart = alt.Chart(n_df).mark_line().encode(
    x = alt.X('Date:T'),
    y= alt.Y('Close')
)
n_df
# nflx_chart

,percent_difference,difference,Date,Low,Open,Volume,High,Close,Adjusted Close,company_id,yr,month,avg_close_by_month
2898850,7.816099,0.097143,23-05-2002,1.145714,1.156429,104790000.0,1.242857,0,1.196429,NFLX,2002,5,0
2898851,2.274046,0.027857,24-05-2002,1.197143,1.214286,11104800.0,1.225000,0,1.210000,NFLX,2002,5,0
2898852,6.086959,0.075000,28-05-2002,1.157143,1.213571,6609400.0,1.232143,0,1.157143,NFLX,2002,5,0
2898853,6.748517,0.078572,29-05-2002,1.085714,1.164286,6757800.0,1.164286,0,1.103571,NFLX,2002,5,0
2898854,3.288148,0.036428,30-05-2002,1.071429,1.107857,10154200.0,1.107857,0,1.071429,NFLX,2002,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2904021,3.221377,10.089996,06-12-2022,303.130005,311.000000,7195400.0,313.220001,0,305.559998,NFLX,2022,6,0
2904022,2.425626,7.509979,07-12-2022,302.100006,306.500000,7821600.0,309.609985,0,308.420013,NFLX,2022,7,0
2904023,2.423333,7.570007,08-12-2022,304.809998,309.109985,7205300.0,312.380005,0,310.260010,NFLX,2022,8,0
2904024,2.881462,9.480011,09-12-2022,319.519989,321.450012,14354700.0,329.000000,0,320.010010,NFLX,2022,9,0
